In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Crew, Task
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI')
genai.configure(api_key=GEMINI_API_KEY)

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.4, google_api_key=GEMINI_API_KEY)

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()